In [17]:
%pip install tqdm
%pip install -q langchain langchain_community 
%pip install pdfplumber




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from tqdm.autonotebook import tqdm
import pdfplumber
import os 
import re
import json

/var/folders/fg/lsnrhm4529g9kh8jm4vfcdrc0000gn/T/ipykernel_27291/2894384040.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:




def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            if i > 0:
                text += "\n"
            text += page.extract_text()
    return text



def remove_unnecessary_newlines(text):
    # Remove lines that contain a number surrounded by whitespace characters on both sides.
    # This likely removes remaining page numbers.
    text = re.sub(r"^\s*\d+\s*\n", "", text, flags=re.MULTILINE)

    # Remove the page number at the very end of the file
    text = re.sub(r"\s*\d+\s*$", "", text)

    # Remove text that starts with a single uppercase letter followed by a dot and a space,
    # and continues until the next number with a dot. This likely removes section headers.
    text = re.sub(r"[A-Z]\.\s*.*?(?=\d+\.)", "", text, flags=re.DOTALL)

    # Remove text that starts from 'Część' (Part), 'Rozdział' (Chapter), or 'Art' (Article)
    # until it reaches the next number with a dot. This likely removes section headers.
    text = re.sub(r"(?:Część|Rozdział|Art)\.?\s*.*?(?=\d+\.)", "", text, flags=re.DOTALL)

    # Replace single newlines with a space. This is likely to merge lines that were
    # split incorrectly during text extraction.
    text = re.sub(r"(?<![?.!])(\n)", " ", text)

    return text

def split_text_by_questions(text):
    # Split text into questions and answers based on question mark
    qa_list = re.split(r"(\d+\.\s.*?\?)\s", text)

    # Remove empty strings from the list
    qa_list = [q for q in qa_list if q.strip()]

    # Initialize list to store question and answer pairs
    result = []
    after_number = re.compile(r'\d+\.\s')

    # Iterate through the list in pairs
    for i in range(1, len(qa_list), 2):
        match = after_number.search(qa_list[i])
        if match:
            question = qa_list[i][match.end():].strip()
        else:
            question = qa_list[i].strip()
        if i + 1 < len(qa_list):
            answer = qa_list[i + 1].strip()
            result.append({"question": question, "answer": answer})
            question= ''
            answer = ''
        else:
            result.append({"question": question, "answer": ""})

    return result
    

In [4]:
file_path = "data/Katechizm.pdf"
pdf_text = extract_text_from_pdf(file_path)



In [5]:

clean_text = remove_unnecessary_newlines(pdf_text)
#print(clean_text)
qa_list = split_text_by_questions(clean_text)
# for qa in qa_list:
 #  print(f"Question: {qa['question']}")
  # print(f"Answer: {qa['answer']}\n")

In [ ]:
def convert_to_jsonl(qa_pairs, filename):
    with open(filename, "w", encoding="utf-8") as f:
        # Write the initial system message as the first line of the file
        initial_message = {
            "messages": [
                {
                    "role": "System",
                    "content": "Jesteś katolickim teologiem uznającym jedynie naukę Kościoła sprzed Soboru Watykańskiego II. Odpowiedz zwieźle na pytania doktrynalne podając odpowiednie źródła."
                },
                {"role": "Chatbot", "content": "Laudatur Jesus Christus!"}
            ]
        }
        json.dump(initial_message, f, ensure_ascii=False)
        f.write("\n")

        # Write the question-answer pairs as subsequent lines of the file
        for qa_pair in qa_pairs:
            messages = [
                {"role": "User", "content": qa_pair["question"]},
                {"role": "Chatbot", "content": qa_pair["answer"]}
            ]
            json.dump({"messages": messages}, f, ensure_ascii=False)
            f.write("\n")

In [8]:
# Convert the question-answer pairs into JSON format and save to a file
convert_to_jsonl(qa_list, "gaspari_data_set.jsonl")